In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
import tensorflow as tf

from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

from keras.applications import MobileNet
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import CSVLogger

import numpy as np
import matplotlib.pyplot as plt

In [3]:
!python --version

Python 3.10.10


In [4]:
!pip install gdown
!gdown --id 1-56qEY2Ctb4cI_GGXP1u7qcukmQT08u1 # 데이터셋 다운로드

















/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.

  warnings.warn(

Downloading...

From (uriginal): https://drive.google.com/uc?id=1-56qEY2Ctb4cI_GGXP1u7qcukmQT08u1

From (redirected): https://drive.google.com/uc?id=1-56qEY2Ctb4cI_GGXP1u7qcukmQT08u1&confirm=t&uuid=199c384f-020b-4a3b-b866-1ab4c7452962

To: /kaggle/working/kfood50_imageSet.zip

100%|███████████████████████████████████████| 2.34G/2.34G [00:14<00:00, 160MB/s]


In [11]:
!unzip /kaggle/working/kfood50_imageSet.zip
!rm kfood50_imageSet.zip

Archive:  /kaggle/working/kfood50_imageSet.zip

   creating: kfood50_imageSet/

   creating: kfood50_imageSet/validation/

   creating: kfood50_imageSet/validation/어묵볶음/

  inflating: kfood50_imageSet/validation/어묵볶음/Img_086_0149.jpg  

  inflating: kfood50_imageSet/validation/어묵볶음/Img_086_0137.jpg  

  inflating: kfood50_imageSet/validation/어묵볶음/Img_086_0210.jpg  

  inflating: kfood50_imageSet/validation/어묵볶음/Img_086_0240.jpg  

  inflating: kfood50_imageSet/validation/어묵볶음/Img_086_0127.jpg  

  inflating: kfood50_imageSet/validation/어묵볶음/Img_086_0170.jpg  

  inflating: kfood50_imageSet/validation/어묵볶음/Img_086_0147.jpg  

  inflating: kfood50_imageSet/validation/어묵볶음/Img_086_0106.jpg  

  inflating: kfood50_imageSet/validation/어묵볶음/Img_086_0108.jpg  

  inflating: kfood50_imageSet/validation/어묵볶음/Img_086_0139.jpg  

  inflating: kfood50_imageSet/validation/어묵볶음/Img_086_0158.jpg  

  inflating: kfood50_imageSet/validation/어묵볶음/Img_086_0193.jpg  

  inflating: kfood50_imageSet/validat

In [12]:
## 이미지셋 로드시 필요한 변수 지정
imgSet_path = '/kaggle/working/kfood50_imageSet/'
train_path = imgSet_path + 'train/'
valid_path = imgSet_path + 'validation/'
test_path = imgSet_path + 'test/'

img_height = 224
img_width = 224
batch_size = 32

food_classes = 50 # 음식 종류 수

In [21]:
datagen = ImageDataGenerator(rescale=1./ 255)

# 학습 데이터셋
train_generator = datagen.flow_from_directory(
    train_path,
    shuffle=True,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# 검증 데이터셋 - 학습시 loss, accuracy 확인 
valid_generator = datagen.flow_from_directory(
    valid_path,
    shuffle=True,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# 테스트 데이터셋 - 학습된 모델 정확도 확인
test_generator = datagen.flow_from_directory(
    test_path,
    shuffle=True,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 37500 images belonging to 50 classes.

Found 7500 images belonging to 50 classes.

Found 5000 images belonging to 50 classes.


In [32]:
def create_model() :

    # 모델 기존 가중치 사용, 가중치 동결 X
    base_model = MobileNet (
                    weights='imagenet', 
                    include_top=False, 
                    input_shape=(224, 224, 3),
                    pooling='avg',
                )

    # 이미지셋에 맞게 output 레이어 추가하기
    x = base_model.output
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    output = Dense(food_classes, activation='softmax')(x)

    # 전체 모델 구성
    model = Model(inputs=base_model.input, outputs=output)

    # 모델 컴파일
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [33]:
model = create_model()
model.summary()

Model: "model_3"

_________________________________________________________________

 Layer (type)                Output Shape              Param #   


 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         

                                                                 

 conv1 (Conv2D)              (None, 112, 112, 32)      864       

                                                                 

 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       

 n)                                                              

                                                                 

 conv1_relu (ReLU)           (None, 112, 112, 32)      0         

                                                                 

 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       

                                                                 

 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       

 ation)                                    

In [34]:
log_path = '/kaggle/working/log/' # 학습 중 loss, accuracy 값 저장 경로
models_path = '/kaggle/working/models/' # 학습된 모델 저장 경로

In [35]:
# CSVLogger를 사용하여 학습 과정 저장
csv_logger = CSVLogger(log_path+'history.log')

history = model.fit(
    train_generator,  # 학습 데이터 제네레이터
    epochs=20,  # 학습 에포크 수
    verbose=1,
    validation_data=valid_generator,  # 검증 데이터 제네레이터
    callbacks = [csv_logger]
)

Epoch 1/20

1172/1172 [==============================] - 146s 122ms/step - loss: 1.5144 - accuracy: 0.5665 - val_loss: 1.5706 - val_accuracy: 0.5621

Epoch 2/20

1172/1172 [==============================] - 149s 127ms/step - loss: 0.9525 - accuracy: 0.7143 - val_loss: 1.5795 - val_accuracy: 0.5764

Epoch 3/20

1172/1172 [==============================] - 149s 127ms/step - loss: 0.7554 - accuracy: 0.7667 - val_loss: 1.1608 - val_accuracy: 0.6731

Epoch 4/20

1172/1172 [==============================] - 140s 119ms/step - loss: 0.6240 - accuracy: 0.8051 - val_loss: 1.6437 - val_accuracy: 0.6089

Epoch 5/20

1172/1172 [==============================] - 141s 120ms/step - loss: 0.5350 - accuracy: 0.8303 - val_loss: 1.2858 - val_accuracy: 0.6659

Epoch 6/20

1172/1172 [==============================] - 151s 128ms/step - loss: 0.4288 - accuracy: 0.8634 - val_loss: 1.4196 - val_accuracy: 0.6456

Epoch 7/20

1172/1172 [==============================] - 142s 121ms/step - loss: 0.3690 - accuracy: 

In [37]:
loss, acc = model.evaluate(test_generator, verbose=2)
print('모델 accuracy: {:5.2f}%'.format(100*acc))
print('모델 accuracy: {:5.2f}%'.format(100*acc))

157/157 - 19s - loss: 1.1874 - accuracy: 0.7708 - 19s/epoch - 118ms/step

모델 정확도: 77.08%


In [39]:
# 학습된 모델 h5 파일로 저장
modelFile = f'MobileNet-32-20-{acc}.h5' # modelName-batch-epochs-accuracy
model.save(models_path+modelFile)

In [40]:
!zip -r history.zip $log_path

  adding: kaggle/working/log/ (stored 0%)

  adding: kaggle/working/log/history0.log (deflated 49%)


In [42]:
!cp /kaggle/working/models/$modelFile /kaggle/working